In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# ENABLE IF USING COLAB
USE_COLAB = True

if USE_COLAB:
    import os
    from google.colab import drive
    
    FOLDERNAME = 'UMass_Amherst/NN_AMI/NN_Project'
    drive.mount('/content/drive')
    %cd /content/drive/My\ Drive/$FOLDERNAME

    if os.path.exists('./spectroconv/'):
      %cd ./spectroconv
      !pip install hub
      !pip install hub[audio]
      !pip install wandb
    else:
      print("Error, must first pull project repository from github to Drive using ColabGitCommands.ipynb")

In [ ]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


In [ ]:
from models import wb_experiment

In [ ]:
n_mels = 128

config = {
  'data': {
    'train': {
        'name': f"nsynth-full-{n_mels}-train"
    },
    'val': {
        'name': f"nsynth-full-{n_mels}-val"
    }
  },
  'group': 'lc',
  'job_type': 'sweep',
  'model': {
      'id': wb_experiment.ModelType.LC,
      'name': 'LC'
  },
}

# wandb's sweep config must be exact, no additional dictionary values are allowed.
wb_sweep_config = {
    'name': 'lc-sweep',
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'method': 'random',
    'metric': {
      'name': 'f1',
      'goal': 'maximize'
    },
    'parameters': {
      'local_dim': {
          'values': [None, 0, 1] # could possibly be split into 3 seperate sweeps
      },
      'kernel_size': {
          'values': [3, 5]
      },
      'num_channels': {
          'distribution': 'int_uniform',
          'min': 10,
          'max': 50,
      },
      'num_conv_layers': {
          'values': [1,2]
      },
      'linear_layer_size': {
          'distribution': 'int_uniform',
          'min': 60,
          'max': 120,
      },
      'learning_rate': {
          'distribution': 'log_uniform_values',
          'min': 0.00005,
          'max': 0.005,
      },
      'dropout': {# probability of an element to be zeroed (0 = no dropout)
          'distribution': 'uniform',
          'min': .40,
          'max': .60,
      },
      'l2_reg': {
          'value': 0, # trying out no l2 regularizer
          #'distribution': 'log_uniform_values',
          #'min': 0.0000001,
          #'max': 0.01,
      },
      'optimizer': {
          'value': 'adam'
      },
      'epochs': {
          'value': 2
      },
      'batch_size': {
          'value': 100
          #'distribution': 'q_log_uniform_values',
          #'q': 4,
          #'min': 72,
          #'max': 128,
      },
      'linear_reg': {
          'value': False
      }
    }
}

config['sweep_config'] = wb_sweep_config

In [ ]:
sweeper = wb_experiment.SweepExperiment(wb_config=config, login_key=True)
sweeper.run_sweep(iterations = 30, save_all = False)